In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision
import torchvision.transforms as transforms
import cv2

In [3]:
import os
import urllib.request
import zipfile

# Define dataset directory
dataset_dir = "../data/kitti_sample/"
zip_filename = "kitti_sample.zip"
zip_path = os.path.join(dataset_dir, zip_filename)
extracted_folder = os.path.join(dataset_dir, "training")  # KITTI usually has 'training' and 'testing' folders

# Create the dataset directory if it doesn't exist
os.makedirs(dataset_dir, exist_ok=True)

# Download if not already downloaded
if not os.path.exists(zip_path):
    print("Downloading KITTI sample dataset...")
    url = "https://s3.eu-central-1.amazonaws.com/avg-kitti/data_object_image_2.zip"
    urllib.request.urlretrieve(url, zip_path)
    print("Download complete.")
else:
    print("Zip file already exists. Skipping download.")

# Extract if not already extracted
if not os.path.exists(extracted_folder):
    print("Extracting ZIP file...")
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(dataset_dir)
    print("Extraction complete.")
else:
    print("Data already extracted. Skipping extraction.")


Zip file already exists. Skipping download.
Data already extracted. Skipping extraction.


In [4]:
import glob
import os

# Tiny dataset path
tiny_dataset_dir = "../data/kitti_sample/training/image_2/"

# Find all images
tiny_image_paths = glob.glob(os.path.join(tiny_dataset_dir, "*.png"))
tiny_image_paths = sorted(tiny_image_paths)

print(f"Total images found in tiny dataset: {len(tiny_image_paths)}")


Total images found in tiny dataset: 7481


In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from utils.dataset import KITTITinyDataset
from models.depth_cnn import SimpleDepthCNN
from torch.utils.data import DataLoader
from torch.utils.data import Subset

# Check device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load dataset
dataset = KITTITinyDataset(
    image_root="../data/kitti_sample/training/image_2/",
    depth_root="../data/kitti_sample/data_depth_annotated/train/"
)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)



# Initialize model
model = SimpleDepthCNN().to(device)

# Define loss and optimizer
criterion = nn.L1Loss()  # Mean Absolute Error (good for depth)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Training loop
# Training loop with loss tracking
num_epochs = 10
losses = []  # Store average loss per epoch

for epoch in range(num_epochs):
    running_loss = 0.0

    for images, targets in dataloader:
        images = images.to(device)
        targets = targets.to(device)

        outputs = model(images)
        outputs = torch.nn.functional.interpolate(outputs, size=targets.shape[-2:], mode="bilinear", align_corners=False)
        loss = criterion(outputs, targets)

        # Backward + optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_loss = running_loss / len(dataloader)
    losses.append(avg_loss)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")


print("Training complete!")

Using device: cpu
📸 Total RGB images before slicing: 7481
💡 Using 98 RGB images starting from index 5
🌊 Using 98 depth maps
Epoch [1/10], Loss: 3.4622
Epoch [2/10], Loss: 3.3461
Epoch [3/10], Loss: 3.2716
Epoch [4/10], Loss: 3.2529
Epoch [5/10], Loss: 3.2723
Epoch [6/10], Loss: 3.2495
Epoch [7/10], Loss: 3.2334
Epoch [8/10], Loss: 3.2224
Epoch [9/10], Loss: 3.2135
Epoch [10/10], Loss: 3.2120
Training complete!
